# 00 - Preparations

In [ ]:
import torch
from ultralytics import YOLO

# 01 - Training

https://docs.ultralytics.com/modes/train/#train-settings

In [ ]:
# Load the YOLO model as a starting point
model = YOLO("yolo11s-seg.pt")

# Train the model on the dataset
results = model.train(
    data="config/config_train.yaml",
    epochs=200,
    patience=0,
    project='models',
    name='yolo11s_250imgs_200epochs',
    device=[-1, -1],
    )

# 02 - Test model on image

In [ ]:
# Load the best model from training
best_model = YOLO("runs/segment/train/weights/best.pt")

In [ ]:
# inference on an image
results = best_model.predict(
    source="test_image.png",
    save=True,
    #show_boxes=False,
    )

# 03 - Prepare model for Depness
https://qgis-plugin-deepness.readthedocs.io/en/latest/

In [ ]:
path_to_model = 'runs/segment/train8/weights/best.pt'

In [ ]:
model = torch.load(path_to_model)
model

In [ ]:
# export the model as ONNX
best_model.export(format="onnx", dynamic=True)

https://qgis-plugin-deepness.readthedocs.io/en/latest/